In [1]:
import pandas as pd
import numpy as np
import pickle

from utils.Sharp import SharpPortfolio
from utils.Markov import MarkovPortfolio
from utils.Tobin import TobinPortfolio

# Disable jedi autocompleter
%config Completer.use_jedi = False

In [2]:
df = pd.read_csv('results/prediction_table.csv', index_col=0)
pct = pd.read_csv('data/ticker_data_preprocessed.csv', index_col=0)
pct = pct.drop(['sector', '2022-07-29', '2022-07-28', '2022-07-27', '2022-07-26', '2022-07-25'], axis=1)
sp500 = pd.read_csv('data/sp500.csv', index_col=0)['Изм. %'][5:]
bonds = pd.read_csv('data/bonds.csv', index_col=0)['Изм. %'][5:]

In [3]:
sp500 = sp500.apply(lambda x: float(x[:-1].replace(',', '.')) / 100)
bonds = bonds.apply(lambda x: float(x[:-1].replace(',', '.')) / 100)

# Обработка даты 

def month(dates : list):
    new_dates = []
    for d in dates:
        date = d.split()
        months = {'янв.' : '01', 'февр.': '02', 'мар.': '03', 'апр.': '04', 'мая': '05',
                  'июн.' : '06', 'июл.': '07', 'авг.': '08', 'сент.': '09', 'окт.': '10', 'нояб.': '11', 'дек.': '12'}
        date = date[0]+'-'+months[date[1]]+'-'+date[2]
        new_dates.append(date)
    return new_dates

sp500.index = month(sp500.index)

## Создаю словари для каждого метода кластеризации

In [4]:
cols = df.columns[1:]

**общий словарь**: ключ - метод кластеризации, значение - словарь(*ключ - номер кластера, значения - тикеры*)

In [5]:
all_accets = {}
for col in cols:
    accets = {}
    for i in set(df[col].unique()):
        accets[i] = list(df.loc[df[col] == i].index.values)
    all_accets[col] = accets

## Выберу из каждого кластера 2 лучшие акции по значению Шарпа и создам портфель для каждого метода кластеризации

In [6]:
def choose_best_accets(dict_accets, df_pct):
    accets_to_port = []
    for cluster in dict_accets.values():
        if len(cluster)==1:
            accets_to_port.append(cluster[0])
        else:
            dict_with_sharp = {}
            sharp = 0
            first_sharp = -100
            second_sharp = -200
            # считаем Шарпа для каждой акции
            for ticker in cluster:
                tickers_pct = df_pct.loc[df_pct.index == ticker].values
                sharp = (tickers_pct.mean()-0.03)/tickers_pct.std()
                dict_with_sharp[sharp] = ticker

            # выбираем 2 лучших акции
            for sharp in dict_with_sharp.keys():
                if first_sharp < sharp:
                    second_sharp = first_sharp
                    first_sharp = sharp
                elif first_sharp > sharp and second_sharp < sharp:
                    second_sharp = sharp
            accets_to_port.append(dict_with_sharp[first_sharp])
            accets_to_port.append(dict_with_sharp[second_sharp])
    return accets_to_port    

In [7]:
# Сохраняю расчеты в словарь: ключ - название портфеля(метод кластеризации), значение - тикеры
ports = {}
for col in cols:
    ports[col] = choose_best_accets(all_accets[col], pct)

## Найду лучшие методы кластеризации исходя из доходности

Веса в портфеле определяются значением коэффициента Шарпа

In [8]:
bonds.index = pd.to_datetime(bonds.index)
sp500.index = pd.to_datetime(sp500.index)

# Настройка размерности

sp500 = sp500.drop(list(set(sp500.index) - set(bonds.index)))
bonds = bonds.drop(list(set(bonds.index) - set(sp500.index)))

In [9]:
pct_monthly = {}
sp500_monthly = {}
bonds_monthly = {}

pct = pct.T[:-1]
pct.index = pd.to_datetime(pct.index)

In [10]:
# Настройка размерности

pct = pct.drop(list(set(pct.index) - set(bonds.index)))
bonds = bonds.drop(list(set(bonds.index) - set(pct.index)))
sp500 = sp500.drop(list(set(sp500.index) - set(pct.index)))

In [11]:
dates = list((sp500.reset_index().groupby(pd.Grouper(key="index", freq="M")).std()).index)

In [12]:
last_date = dates[0]
pct_monthly[last_date] = pct[(pct.index<=last_date)]
sp500_monthly[last_date] = sp500[(sp500.index<=last_date)]
bonds_monthly[last_date] = bonds[(bonds.index<=last_date)]
for date in dates[1:]:
    pct_monthly[date] = pct[((pct.index>last_date)&(pct.index<=date))]
    sp500_monthly[date] = sp500[((sp500.index>last_date)&(sp500.index<=date))]
    bonds_monthly[date] = bonds[((bonds.index>last_date)&(bonds.index<=date))]
    last_date=date

Для каждого портфеля создаю словарь с доходностями

In [13]:
def create_dict_monthly(dates, pct_monthly, tickers):
    d = {}
    for date in dates:
        d[date] = pct_monthly[date][tickers]
    return d

In [14]:
def create_dict_half_year(dates, pct_monthly, tickers):
    d = {}
    for date in range(6, len(dates)):
        d[dates[date]] = pd.concat([pct_monthly[dates[date-1]][tickers], 
                                   pct_monthly[dates[date-2]][tickers],
                                   pct_monthly[dates[date-3]][tickers], 
                                   pct_monthly[dates[date-4]][tickers],
                                   pct_monthly[dates[date-5]][tickers],
                                   pct_monthly[dates[date-6]][tickers]])
    return d

In [15]:
port_names = list(ports.keys())
portfolios = {}
for port in port_names:
    portfolios[port] = create_dict_monthly(dates, pct_monthly, ports[port])

In [16]:
pct_half_year = {}
for port in port_names:
    pct_half_year[port] = create_dict_half_year(dates, pct_monthly, ports[port])

In [17]:
dates[6]

Timestamp('2018-07-31 00:00:00', freq='M')

In [18]:
pct_half_year['original_n'][dates[6]]

,FCX,MOS,TWTR,MTCH,CZR,PENN,NWL,SYY,APA,OXY,...,ALGN,DXCM,AAL,UAL,SPG,VTR,ENPH,SEDG,AES,NRG
2018-06-04,0.005257,-0.007231,0.033561,-0.005636,-0.007709,-0.003252,0.004325,0.005045,0.005389,0.003055,...,0.030352,0.013666,0.019545,0.010542,0.011318,0.002735,0.001730,0.023766,0.010946,-0.001769
2018-06-06,0.031162,0.043416,0.007538,-0.004441,0.019101,0.021759,0.022053,-0.000760,0.008408,0.011110,...,0.000838,-0.002542,0.000226,0.015003,0.014362,-0.002936,-0.056939,0.004695,-0.032483,0.008618
2018-06-07,-0.014835,0.007565,-0.009975,-0.040385,-0.025358,-0.002366,0.015353,-0.000304,0.054826,0.000231,...,-0.020354,-0.049899,-0.007920,-0.011654,-0.002904,0.004049,0.003774,-0.000935,0.016787,-0.012964
2018-06-08,0.005577,0.005461,0.038035,0.015904,-0.016968,-0.039431,0.008582,0.005931,0.011737,-0.011679,...,0.021661,0.039222,-0.016651,0.006471,0.003580,0.005132,-0.054511,0.008419,0.003931,-0.010448
2018-06-11,0.008874,-0.000679,0.005096,0.013488,-0.002302,-0.003086,0.036872,0.003930,0.055871,0.007956,...,0.005106,0.021613,0.024356,0.030862,-0.001693,0.001276,-0.081511,-0.018553,0.004698,-0.014781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-01-25,0.011224,-0.018750,-0.009388,0.013868,-0.014948,0.016005,-0.205572,0.004080,-0.013923,-0.010572,...,0.004701,0.018705,-0.031758,-0.043012,-0.011439,0.009304,0.034653,0.031162,0.029570,0.014679
2018-01-26,-0.014127,0.034845,0.095217,0.021621,0.003035,-0.011209,0.020959,0.002032,0.013277,0.005734,...,0.038054,-0.012581,0.000377,0.007264,-0.002984,-0.009218,0.023924,0.009615,0.020017,-0.009042
2018-01-29,0.004606,0.002896,0.037495,0.012093,-0.031770,-0.011642,0.001184,-0.011385,-0.029742,-0.012179,...,-0.025355,-0.007576,-0.007349,-0.011719,-0.004826,-0.006441,-0.051402,0.009524,0.007679,-0.019708
2018-01-30,-0.025471,-0.018412,0.017474,-0.011949,0.007812,-0.007750,0.014590,-0.001420,-0.044587,-0.019412,...,-0.024172,0.028973,-0.001708,0.018699,-0.001903,-0.015847,0.000000,-0.039083,-0.016935,-0.028667


В переменной portfolios хранятся словари(каждый метод кластеризации) с доходностями своих тикеров помесячно    
В переменной pct_half_year хранятся словари(каждый метод кластеризации) с доходностями своих тикеров за полгода

### Расчет портфелей

1) Выбираем метод кластеризацци
2) Считаем для него оптимизацию

In [19]:
def calc_ports(test, riskless_ret, index, mu, sigma):
    sharp_values, markov_values, tobin_values, traynor_values, qsharp_values = [], [], [], [], []
    # Расчет Шарпа
    sharp = SharpPortfolio(mu, sigma, riskless_ret.mean())
    sh_values = test @ sharp.fit()
    for value in sh_values:
        sharp_values.append(value)

    # Расчет Марковица
    markov = MarkovPortfolio(mu, sigma, index)
    m_values = test @ markov.fit()
    for value in m_values:
        markov_values.append(value)

    # Расчет Тобина

    tobin = TobinPortfolio(mu, sigma, riskless_ret.mean(), index.std())
    tob_values = test @ tobin.fit()
    for value in tob_values:
        tobin_values.append(value)


    df_all_ports = pd.DataFrame({'date':test.index,
                                 'sharp':sharp_values,
                                 'markov':markov_values,
                                 'tobin':tobin_values})
    return df_all_ports

In [20]:
dict_ports = {}
for port in list(portfolios.keys()):
    cluster = portfolios[port]
    test = cluster[dates[6]]
    riskless_ret = bonds_monthly[dates[6]]
    index = sp500_monthly[dates[6]]
    
    half_year = pct_half_year[port][dates[6]]
    mu = half_year.mean().values
    sigma = half_year.cov().values

    dict_ports[port] = calc_ports(test, riskless_ret, index, mu, sigma)
    for date in dates[7:]:
        test = cluster[date]
        riskless_ret = bonds_monthly[date]
        index = sp500_monthly[date]
        
        half_year = pct_half_year[port][date]
        mu = half_year.mean().values
        sigma = half_year.cov().values
        layer = calc_ports(test, riskless_ret, index, mu, sigma)
        dict_ports[port] = pd.concat([dict_ports[port], layer])

-0.18363962600439443 0.09553931418252
0.04314475067480906 0.08688130430105115
0.020244805533316628 0.17354599484816563
0.12043483796347265 0.19878889418294823
0.04275868211076441 0.03524286688912212
0.1465320409632577 0.02276590506180405
0.16905480134112683 -0.05828874214348263
0.12658363601633904 0.016789843905207102
0.4769098814347454 -0.006343624398066895
-0.01642107851802769 -0.03490850559370077
0.2653193292950516 0.03937372597749666
0.28609245132742317 -0.04061307861165393
0.08327090754429049 0.08045379676233161
-0.06498254941443853 -0.038408772938264425
0.11977043221265347 -0.04239297656060051
0.3578003419384937 0.015008100237215554
-0.16735054183118334 0.00908116871697574
-0.060809106460041513 0.10997293813768375
0.5288385881823138 0.14142139966226755
2.8027591976320534 0.10974530956125159
2.112080901188658 0.042158720105985856
0.6555351078575221 -0.07011742237566215
-0.202218819049304 -0.0015676449740841653
0.1143122803542363 0.01716067962066658
0.1922307334213497 0.02195768195

0.08633424332742035 -0.00693421091625491
0.42408702773821166 -0.00800941617019714
-0.029224734665791584 -0.03992990352745735
0.22547607029609076 0.019515139612338393
0.21927696756301848 -0.06546198834048708
0.033550188112287535 0.03692908032104824
-0.09970135943111504 -0.06621313456053893
0.032888582051824995 -0.1104896555875535
0.23295181780909452 -0.06624529948643605
-0.18896275104729315 -0.034004080172759826
-0.06439050615641494 0.08934691027696538
0.4494501640550565 0.10379805394402716
2.575075033750119 0.10365728203375553
2.1399373461226916 0.06332011541160039
0.6906128557226812 -0.06264118421282087
-0.20580278466951005 0.0027133546954602864
0.11942218970798457 0.020803283181278876
0.2026209554805837 0.025629680251864597
-0.16196798715327923 0.0385003761595859
0.4288018373492365 0.08771601527836334
0.4091130681192928 0.24478088853306845
0.9874649996738628 0.14951141401661613
-0.07811949586892157 0.1841965683478899
0.9133862511335679 0.22408388944055113
-0.745259483664883 0.1905055

0.05978793305532749 -0.0851533398431812
0.26606816587841897 -0.039329016030332856
-0.1854065477133767 -0.02749738836779971
-0.09103373950298639 0.06279569336795399
0.4475002008195649 0.10038982690430263
2.6043267236848706 0.10879699938576215
2.092726809192922 0.07485889540503765
0.6558665789439722 -0.06291928764940972
-0.1942442080420044 0.00742244922731597
0.11852622838892601 0.02526797101434668
0.19407591358556647 0.026694101999485017
-0.14613655767215109 0.04348092180576693
0.40588046428264224 0.08442552166829721
0.39755431676627 0.24445859384297122
0.9388362998066037 0.16444329560428508
-0.048565804293754976 0.20499111721381516
0.8886906523268422 0.24574259755013894
-0.675896252660646 0.20559383036877488
-0.3765283118138823 0.17003849029773
1.0098105558871147 0.16353517036147905
0.8018846622316598 0.20827176044368775
-1.3679046344422905 0.12496569266090363
0.1116500469072797 0.12490395585952999
0.7019656246971768 0.12209168760032128
-1.1917501825616035 0.09071472565358155
-1.074910

-0.20353332419830514 0.0020893488933785573
0.12543572365809308 0.0255202976232002
0.20644970476732047 0.033339820283620425
-0.1542697947027951 0.04297596937099725
0.41625285955235164 0.08283571916345103
0.4267971899536764 0.25703161425310816
1.0035139778197864 0.18010823674819676
-0.06162146426913271 0.20386787216580507
0.9072333212576971 0.2395586189874282
-0.6835570018712309 0.20699146578882646
-0.392307326215217 0.16614148078762997
1.028673419234991 0.15843716722200374
0.8136455500524221 0.20211327512061483
-1.4111485040270055 0.1295880683597002
0.11431797404614734 0.12920350251558976
0.7262049813973454 0.10730609898034861
-1.303127102889408 0.07067646066904455
-1.210293187783469 0.07505856802852089
-0.18592853045828556 0.08958583946334976
-1.1796015633314008 0.047085816172098664
-1.7755861661399257 0.015710279302831774
-0.3575945831252039 0.02726605854174779
-1.51960863669828 0.00972269357745227
-0.18766747724940028 0.051859469943879306
-0.40404028748565507 -0.0031105172104901394
-

-1.009843313937994 0.19657003853238975
-0.6347809672384124 0.11510368779597548
1.330412962865354 0.11570013052952921
1.0252071847228852 0.18978088616937017
-1.7029972057228202 0.13317291774391177
0.10071737192763808 0.12074422712580692
0.8449262493659828 0.12212374219780298
-1.4426169025725473 0.12749125380658763
-1.412561588686451 0.10760080129235026
-0.19975558481705982 0.1431489653444064
-1.5256593537955074 0.05442558266130853
-2.207444676204538 0.01931538769807579
-0.4576739141290993 -0.014523355125106486
-1.7600911650126823 -0.07485844047914218
-0.24188857620391055 0.0064877412837733356
-0.46028397977342156 -0.05773210237513305
-0.589489705711026 0.010196853534215972
-0.14400962363687073 -0.024443370641908182
-0.23044958238618213 0.03711572933167531
-0.03740428193723009 0.012898001804157164
-0.10849006100744452 0.037815352271379506
-0.00905244223486991 0.07229619377483743
-0.030611886294012246 -0.029283295859001912
0.11893705838293013 0.009054724626255726
0.1447298130851398 -0.072

...

D:\DS_IT\Anaconda\lib\site-packages\scipy\optimize\optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


-1.0161681207724786 0.07186209982738893
-0.15839815668184262 0.0734158492303088
-1.0552768875693752 0.018243976188849192
-1.5572668098381752 -0.001396846549632413
-0.3423086076395262 0.0009508496001510258
-1.3649581949899707 -0.0243977463772642
-0.2092834304175777 -0.007856749207400796
-0.3635417345565214 -0.027947142501693834
-0.4836616596573854 0.008617309214523612
-0.1293838440579241 -0.031166709959715076
-0.18672946495154544 0.06901275432572576
-0.0012858444575976548 0.048674293815343515
-0.05995726378173812 0.08506942101431242
0.032068683955612166 0.1135994495678625
-0.005531189816019946 -0.0028956570255403782
0.09248564844732164 -0.0036138745973985868
0.10369353953364903 -0.0901465638367235
0.07366305682961255 -0.017269391452238176
0.4134171264240628 -0.0031449232869916266
-0.01567649391516774 -0.02643125867699379
0.22917554544633856 0.02714069356291581
0.22891800661973247 -0.04923974405935987
0.0614288915900543 0.06387417840971074
-0.06897902362037775 -0.04281270638997936
0.0403

-0.31907494413065973 -0.0004966467672540558
-0.41408190453194077 0.04480019300433728
-0.07993390389055076 0.006139415756653057
-10000000 0.17891752114890663
-10000000 0.16984757769415196
-10000000 0.12104557589630188
-10000000 0.11029075264188182
-10000000 0.03006066589691836
-10000000 0.003988268672959698
-10000000 -0.05376087214847441
-10000000 0.03810670944116493
-10000000 0.07974703320770352
-10000000 0.049224722478880406
-10000000 0.12890738340057234
-10000000 0.1252622399111089
-10000000 0.19461616250412317
-10000000 0.13657077851825022
-10000000 0.08368929790316688
-10000000 0.0728407187459336
-10000000 0.05294140254204306
-10000000 0.07130981942075341
-10000000 0.12669660006651443
-10000000 0.07903448245027808
-10000000 0.11176108886890998
-10000000 -0.004697326710375579
-10000000 0.06071547436456575
-10000000 0.08685395306426888
-10000000 0.06452218577106487
-10000000 0.05896398484726266
-10000000 0.07917987519880336
-10000000 0.24408993025612083
-10000000 0.17761677654732266


0.508673830140117 0.09884200725915086
2.9809301762575524 0.10245696258909914
2.338277590725109 0.0665472757087159
0.7445610117530839 -0.06889411358178715
-0.2248990510551382 0.002830687561303155
0.12805624125663997 0.022109678056260677
0.21459718129345665 0.02395358291961734
-0.18478405899192646 0.03507524204961295
0.4417011164483803 0.07239272381475798
0.4087708189750166 0.23075261715938247
1.060955570467908 0.13142186262293531
-0.11348787024491237 0.17859813227358617
0.9572287391007569 0.2103231047003834
-0.7837164867479399 0.19633339619257148
-0.44553156781595993 0.16868948623438038
1.1487252431016926 0.1646440826183183
0.9057216395606319 0.21583825615531835
-1.5742347145078228 0.14520048507810365
0.11787019803788285 0.13425276173186126
0.7582459009653126 0.09651254552267845
-1.3594724417280541 0.06667045904925482
-1.1931057843167905 0.07892702249811992
-0.17640280307853562 0.09665430484375855
-1.2120039525476571 0.04234083887782936
-1.8250707971342854 0.02537957027119458
-0.3663753

-0.1338078005098808 0.043023560727429785
0.3854279192744175 0.08955494371150058
0.4038258735169116 0.26029205614217354
0.896669103693774 0.1733609665953127
-0.022060846417591956 0.21191413726888036
0.842576067332671 0.26214803241264795
-0.5408590609979648 0.22906960651927855
-0.28059482423131 0.19210290174081723
0.9139294988223137 0.17967215558400845
0.7348732567153821 0.2295456841640104
-1.0798823774640327 0.14847931732968941
0.12367040942673944 0.1383091459768849
0.6138493953176225 0.12556291447663787
-1.0132428167444174 0.0775724948139523
-0.9362628310521395 0.10328500404389826
-0.10994417735379289 0.12138902213926742
-0.9925011762176675 0.05775371432315309
-1.4040811581761052 0.028241188768186716
-0.28497498664883025 0.024815238854279477
-1.211235942289217 0.011398162157027924
-0.15072160500102083 0.03934407462667508
-0.3311271932358882 -0.0105722541381972
-0.4180955597097897 0.043063373710411315
-0.10227757107822046 -0.01061102306036718


In [21]:
df_res_sharp = pd.DataFrame([])
df_res_markov = pd.DataFrame([])
df_res_tobin = pd.DataFrame([])
columns = []

for i, (k, v) in enumerate(dict_ports.items()):
    col_names = v.columns
    df_loc_sharp = pd.DataFrame(data=v[col_names[1]].values, index=v[col_names[0]].values, columns = [k])
    df_res_sharp = pd.concat([df_res_sharp, df_loc_sharp], axis=1)
    df_loc_markov = pd.DataFrame(data=v[col_names[2]].values, index=v[col_names[0]].values, columns = [k])
    df_res_markov = pd.concat([df_res_markov, df_loc_markov], axis=1)
    df_loc_tobin = pd.DataFrame(data=v[col_names[3]].values, index=v[col_names[0]].values, columns = [k])
    df_res_tobin = pd.concat([df_res_tobin, df_loc_tobin], axis=1)
    columns.append(columns)

In [22]:
df_res_sharp['sp500']=sp500
df_res_markov['sp500']=sp500
df_res_tobin['sp500']=sp500

df_res_sharp.to_csv('results/df_res_sharp.csv')
df_res_markov.to_csv('results/df_res_markov.csv')
df_res_tobin.to_csv('results/df_res_tobin.csv')

In [23]:
df_real_sharp = (df_res_sharp + 1).cumprod()
df_real_markov = (df_res_markov + 1).cumprod()
df_real_tobin = (df_res_tobin + 1).cumprod()

df_real_sharp.to_csv('results/df_real_sharp.csv')
df_real_markov.to_csv('results/df_real_markov.csv')
df_real_tobin.to_csv('results/df_real_tobin.csv')

## Метрики Sharp

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import dates as mdates

from scipy.stats import norm

import functools

## Поиск метрик

In [25]:
df_all_ports_pct = pd.read_csv('results/df_res_sharp.csv', index_col=0)
df_all_ports_pct.index = pd.to_datetime(df_all_ports_pct.index)
days = df_all_ports_pct.index

## Доходность

In [26]:
ret_ports = df_all_ports_pct.mean()#находим доходности портфелей

## Безрисковый актив

In [27]:
df_b = pd.read_csv(r'data/bonds.csv', decimal=",")
df_bond = pd.DataFrame()

# Найти среднюю цену за месяц и поделить её на 100
df_bond['date'] = df_b['Дата']
df_bond['цена'] = df_b['Цена']/100
df_bond.date = pd.to_datetime(df_bond.date)
df_bond.set_index('date', inplace = True)

df_bond = df_bond.mean()

## Риск портфеля

In [28]:
risk_ports = df_all_ports_pct.std()

## Beta портфеля

In [29]:
var_ports = df_all_ports_pct.var()
cov_ports = df_all_ports_pct.cov()

In [30]:
# Beta = cov(ret_port;ret_IMOEX)/var(IMOEX), beta считается, только для портфелей, не для индекса
beta = cov_ports['sp500']/var_ports['sp500']
beta = beta.to_frame()
beta_ports = (cov_ports['sp500']/var_ports['sp500']).to_frame().T.rename({'sp500':'Beta'})

## Коэффициент VaR

In [31]:
mean_for_var = df_all_ports_pct.mean()
std_for_var = df_all_ports_pct.std()

In [32]:
l_pr_values = df_all_ports_pct.iloc[-1, :].values

In [33]:
var = l_pr_values*(1+norm.ppf(0.001, mean_for_var, std_for_var))/l_pr_values-1
var = pd.DataFrame(data=var.reshape(1,-1), columns=mean_for_var.index, index=['VaR'])
var

,original_n,deco_pca_Kmean,deco_pca_Agglo,deco_pca_MiniB,deco_pca_Gauss,deco_pca_Spect,deco_fast_Kmean,deco_fast_Agglo,deco_fast_MiniB,deco_fast_Gauss,...,neur_lstm_Gauss,neur_lstm_Spect,orig__Kmean,orig__Agglo,orig__MiniB,orig__Gauss,orig__Spect,orig__origi,orig__rando,sp500
VaR,-0.073099,-0.072941,-0.086899,-0.068909,-0.071966,-0.076654,-0.073532,-0.081765,-0.075231,-0.077019,...,-0.125835,-0.088104,-0.069313,-0.092476,-0.065519,-0.067204,-0.077554,-0.073099,-0.083227,-0.10566


## Коэффициент Шарпа

In [34]:
std_ports = df_all_ports_pct.std()

sharp_ports = (ret_ports - df_bond['цена'])/std_ports

## Просадка и восстановление

In [35]:
def find_max_drawdown(prices):

    max_price = prices.iloc[0]
    curr_drawdown = 0
    max_drawdown = 0
    curr_left = 0
    left = 0
    right = 0
    for i in range(0, len(prices)):
        curr_drawdown = (prices.iloc[i] / max_price - 1) * 100
        # print(type(curr_drawdown))
        if curr_drawdown < max_drawdown:
            max_drawdown = curr_drawdown
            left = curr_left
            right = i
        if prices.iloc[i] > max_price:
            max_price = prices.iloc[i]
            curr_left = i
    return max_drawdown

In [36]:
def find_max_recovery(prices):
    
    def calc_growth(prices):

        growth = []
        past_p = 0
        for p in prices:
            if past_p:
                growth.append(p - past_p)
            past_p = p
        return growth

    growth = calc_growth(prices)
    s = 0
    left = 0
    right = 0
    curr_left = 0
    max_recovery = 0
    for i in range(0, len(growth)):
        if not s:
            curr_left = i
        s += growth[i]
        if s > 0:
            s = 0
            if max_recovery < (i - curr_left):
                max_recovery = i - curr_left
                left = curr_left
                right = i

    return max_recovery

In [37]:
recovery = []
drawdown = []
for i in range(len(df_all_ports_pct.columns)):
    recovery.append(find_max_recovery(df_all_ports_pct.iloc[:,i]))
    drawdown.append(find_max_drawdown(df_all_ports_pct.iloc[:,i]))

In [38]:
risk = pd.DataFrame(risk_ports, columns =['Risk']).T
ret = pd.DataFrame(ret_ports, columns =['Ret']).T
sharp = pd.DataFrame(sharp_ports, columns =['Sharp']).T
rec = pd.Series(recovery)
rec = pd.DataFrame(rec, columns =['Recovery']).T
drawd = pd.Series(drawdown)
drawd = pd.DataFrame(drawd, columns =['Drawdown']).T

In [39]:
all_metrics = pd.concat([var,beta_ports, risk, ret, sharp])

rec.set_axis(list(all_metrics.columns), axis=1, inplace=True)
drawd.set_axis(list(all_metrics.columns), axis=1, inplace=True)

all_metrics = all_metrics.append(rec)
all_metrics = all_metrics.append(drawd)

all_metrics = all_metrics.T
all_metrics['Method'] = ['Sharp']*len(all_metrics)

## Метрики Markov

## Поиск метрик

In [40]:
df_all_ports_pct = pd.read_csv('results/df_res_markov.csv', index_col=0)
df_all_ports_pct.index = pd.to_datetime(df_all_ports_pct.index)
days = df_all_ports_pct.index

## Доходность

In [41]:
ret_ports = df_all_ports_pct.mean()#находим доходности портфелей

## Риск портфеля

In [42]:
risk_ports = df_all_ports_pct.std()

## Beta портфеля

In [43]:
var_ports = df_all_ports_pct.var()
cov_ports = df_all_ports_pct.cov()

In [44]:
# Beta = cov(ret_port;ret_IMOEX)/var(IMOEX), beta считается, только для портфелей, не для индекса
beta = cov_ports['sp500']/var_ports['sp500']
beta = beta.to_frame()
beta_ports = (cov_ports['sp500']/var_ports['sp500']).to_frame().T.rename({'sp500':'Beta'})

## Коэффициент VaR

In [45]:
mean_for_var = df_all_ports_pct.mean()
std_for_var = df_all_ports_pct.std()

In [46]:
l_pr_values = df_all_ports_pct.iloc[-1, :].values

In [47]:
var = l_pr_values*(1+norm.ppf(0.001, mean_for_var, std_for_var))/l_pr_values-1
var = pd.DataFrame(data=var.reshape(1,-1), columns=mean_for_var.index, index=['VaR'])
var

,original_n,deco_pca_Kmean,deco_pca_Agglo,deco_pca_MiniB,deco_pca_Gauss,deco_pca_Spect,deco_fast_Kmean,deco_fast_Agglo,deco_fast_MiniB,deco_fast_Gauss,...,neur_lstm_Gauss,neur_lstm_Spect,orig__Kmean,orig__Agglo,orig__MiniB,orig__Gauss,orig__Spect,orig__origi,orig__rando,sp500
VaR,-0.09346,-0.094134,-0.095374,-0.091441,-0.087747,-0.094662,-0.090804,-0.094908,-0.088038,-0.090705,...,-0.067222,-0.096279,-0.09369,-0.097329,-0.093084,-0.092724,-0.100248,-0.09346,-0.090263,-0.10566


## Коэффициент Шарпа

In [48]:
std_ports = df_all_ports_pct.std()
sharp_ports = (ret_ports - df_bond['цена'])/std_ports

## Просадка и восстановление

In [49]:
recovery = []
drawdown = []
for i in range(len(df_all_ports_pct.columns)):
    recovery.append(find_max_recovery(df_all_ports_pct.iloc[:,i]))
    drawdown.append(find_max_drawdown(df_all_ports_pct.iloc[:,i]))

In [50]:
risk = pd.DataFrame(risk_ports, columns =['Risk']).T
ret = pd.DataFrame(ret_ports, columns =['Ret']).T
sharp = pd.DataFrame(sharp_ports, columns =['Sharp']).T
rec = pd.Series(recovery)
rec = pd.DataFrame(rec, columns =['Recovery']).T
drawd = pd.Series(drawdown)
drawd = pd.DataFrame(drawd, columns =['Drawdown']).T

In [51]:
markov_metrics = pd.concat([var,beta_ports, risk, ret, sharp])

rec.set_axis(list(markov_metrics.columns), axis=1, inplace=True)
drawd.set_axis(list(markov_metrics.columns), axis=1, inplace=True)

markov_metrics = markov_metrics.append(rec)
markov_metrics = markov_metrics.append(drawd)

markov_metrics = markov_metrics.T
markov_metrics['Method'] = ['Markov']*len(all_metrics)

In [52]:
all_metrics = pd.concat([all_metrics, markov_metrics])

## Метрики Tobin

## Поиск метрик

In [53]:
df_all_ports_pct = pd.read_csv('results/df_res_tobin.csv', index_col=0)
df_all_ports_pct.index = pd.to_datetime(df_all_ports_pct.index)
days = df_all_ports_pct.index

## Доходность

In [54]:
ret_ports = df_all_ports_pct.mean()#находим доходности портфелей

## Риск портфеля

In [55]:
risk_ports = df_all_ports_pct.std()

## Beta портфеля

In [56]:
var_ports = df_all_ports_pct.var()
cov_ports = df_all_ports_pct.cov()

In [57]:
# Beta = cov(ret_port;ret_IMOEX)/var(IMOEX), beta считается, только для портфелей, не для индекса
beta = cov_ports['sp500']/var_ports['sp500']
beta = beta.to_frame()
beta_ports = (cov_ports['sp500']/var_ports['sp500']).to_frame().T.rename({'sp500':'Beta'})

## Коэффициент VaR

In [58]:
mean_for_var = df_all_ports_pct.mean()
std_for_var = df_all_ports_pct.std()

In [59]:
l_pr_values = df_all_ports_pct.iloc[-1, :].values

In [60]:
var = l_pr_values*(1+norm.ppf(0.001, mean_for_var, std_for_var))/l_pr_values-1
var = pd.DataFrame(data=var.reshape(1,-1), columns=mean_for_var.index, index=['VaR'])
var

,original_n,deco_pca_Kmean,deco_pca_Agglo,deco_pca_MiniB,deco_pca_Gauss,deco_pca_Spect,deco_fast_Kmean,deco_fast_Agglo,deco_fast_MiniB,deco_fast_Gauss,...,neur_lstm_Gauss,neur_lstm_Spect,orig__Kmean,orig__Agglo,orig__MiniB,orig__Gauss,orig__Spect,orig__origi,orig__rando,sp500
VaR,-0.108854,-0.118511,-0.118491,-0.122004,-0.122197,-0.118511,-0.118647,-0.113164,-0.120864,-0.118676,...,-0.118804,-0.118443,-0.118904,-0.113134,-0.110828,-0.110149,-0.116996,-0.108854,-0.112757,-0.10566


## Коэффициент Шарпа

In [61]:
std_ports = df_all_ports_pct.std()
sharp_ports = (ret_ports - df_bond['цена'])/std_ports

## Просадка и восстановление

In [62]:
recovery = []
drawdown = []
for i in range(len(df_all_ports_pct.columns)):
    recovery.append(find_max_recovery(df_all_ports_pct.iloc[:,i]))
    drawdown.append(find_max_drawdown(df_all_ports_pct.iloc[:,i]))

In [63]:
risk = pd.DataFrame(risk_ports, columns =['Risk']).T
ret = pd.DataFrame(ret_ports, columns =['Ret']).T
sharp = pd.DataFrame(sharp_ports, columns =['Sharp']).T
rec = pd.Series(recovery)
rec = pd.DataFrame(rec, columns =['Recovery']).T
drawd = pd.Series(drawdown)
drawd = pd.DataFrame(drawd, columns =['Drawdown']).T

In [64]:
tobin_metrics = pd.concat([var,beta_ports, risk, ret, sharp])

rec.set_axis(list(tobin_metrics.columns), axis=1, inplace=True)
drawd.set_axis(list(tobin_metrics.columns), axis=1, inplace=True)

tobin_metrics = tobin_metrics.append(rec)
tobin_metrics = tobin_metrics.append(drawd)

tobin_metrics = tobin_metrics.T
tobin_metrics['Method'] = ['Tobin']*len(tobin_metrics)

In [65]:
all_metrics = pd.concat([all_metrics, tobin_metrics])

## Сохранение результата

In [66]:
all_metrics.to_excel('results/metrics.xlsx')